<a href="https://colab.research.google.com/github/shikarukitake/weather_tg_bot/blob/master/telegrambot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pytelegrambotapi

In [0]:
import telebot

def weather_get_byAPI(city, country=None):
    import requests

    # API KEY
    API_key = "9c82f8d680922dfbfeeb2602c42d70bc"
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    # Creating city name for request
    if country == None:
        city_name = city
        Final_url = base_url + "appid=" + API_key + "&q=" + city_name + '&units=metric'
    else:
        city_name = city + ',' + country
        Final_url = base_url + "appid=" + API_key + "&q=" + city_name + '&units=metric'

    # Request
    weather_data = requests.get(Final_url).json()

    # Obtaining request
    icon_url = "http://openweathermap.org/img/w/" + weather_data['weather'][0]['icon'] + ".png"
    weather_string = f"Temperature is {weather_data['main']['temp']}"
    weather_string += f"\nMax temperature is {weather_data['main']['temp_max']}"
    weather_string += f"\nMin temperature is {weather_data['main']['temp_min']}"
    weather_string += f"\nFeels like {weather_data['main']['feels_like']}"
    weather_string += f"\nWeather is {weather_data['weather'][0]['description']}"
    return weather_string

# Database))00
user_dict = {} 
current_user_dict = {}
exceptions = []

In [0]:
token = input("TelegramBotToken")

In [0]:
bot = telebot.TeleBot(token)

keyboard1 = telebot.types.ReplyKeyboardMarkup()
keyboard1.row('Yes', 'No!')
markup = telebot.types.ForceReply(selective=False)
markupHide = telebot.types.ReplyKeyboardRemove(selective=False)

# Welcome
@bot.message_handler(commands=['start'])
def send_welcome(message):
    try:
        msg = bot.reply_to(message, 'Input city name')
        bot.register_next_step_handler(msg, user_reg)
    except Exception as e:
        bot.reply_to(message, 'ooopps')

@bot.message_handler(commands=['info'])
def send_info(message):
    try:
        chat_id = message.chat.id
        city_string = f"Your city is {user_dict[chat_id]}"
        bot.reply_to(message, city_string)
    except Exception as e:
        bot.reply_to(message, 'ooopps')

# Get weather of current city
@bot.message_handler(commands=['weather'])
def current_weather(message):
    try:
        chat_id = message.chat.id
        city = user_dict[chat_id]
        weather(chat_id, city)
    except Exception as e:
        bot.reply_to(message, "Some error")

# Get weather of some city
@bot.message_handler(commands=['anotherweather'])
def another_weather(message):
    try:
        chat_id = message.chat.id
        msg = bot.send_message(chat_id, 'Input city name', reply_markup=markup)
        bot.register_next_step_handler(msg, another_weather_country)
    except Exception as e:
        bot.reply_to(message, "Some error /anotherweather")

# Selecting manage with country or not
def another_weather_country(message):
    try:
        chat_id = message.chat.id
        current_user_dict[chat_id] = message.text
        msg = bot.send_message(chat_id, 'Do u want to manage country?', reply_markup=keyboard1)
        bot.register_next_step_handler(msg, another_weather_next)
    except Exception as e:
        bot.reply_to(message, "Some error country manage")

# Manage county
def another_weather_next(message):
    try:
        chat_id = message.chat.id
        if message.text == 'No!':
            weather(chat_id, current_user_dict[chat_id])
        elif message.text == 'Yes':
            help_string = 'Input country name in two letter format (ISO 3166)\nFor example: Russian Federation is RU'
            msg = bot.send_message(chat_id, help_string, markup=markupHide)
            bot.register_next_step_handler(msg, weather_with_country)
    except Exception as e:
        bot.reply_to(message, "Some error")

def weather_with_country(message):
    try:
        chat_id = message.chat.id
        country = message.text
        weather(chat_id, current_user_dict[chat_id], country)
    except Exception as e:
        bot.reply_to(message, "Some error")   
   
# Get weather of city
def weather(chat_id, city, country=None):
    try:
        weather = weather_get_byAPI(city, country)
        bot.send_message(chat_id, weather, reply_markup=markupHide)
    except Exception as e:
        bot.send_message(chat_id, "I can't get weather of this city")

# Setting city of user
def user_reg(message):
    try:
        chat_id = message.chat.id
        user_dict[chat_id] = message.text
        msg = bot.reply_to(message, 'Do u want to know weather in ' + user_dict[chat_id] + '?', reply_markup = keyboard1)
        bot.register_next_step_handler(msg, yes_or_no)
    except Exception as e:
        bot.reply_to(message, 'ooopps')

def yes_or_no(message):
    try:
        chat_id = message.chat.id
        if message.text == 'Yes':
            current_weather(message)
        else:
            bot.send_message(chat_id, 'Goodbye!', reply_markup=markupHide)
    except Exception as e:
        bot.reply_to(message, 'ooopps')

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    try:
        chat_id = message.chat.id
        if chat_id in user_dict:
            help_string = '/weather - to get weather of your city\n'
            help_string += '/anotherweather - to get weather of some city\n'
            help_string += '/start - to change your city\n'
            help_string += '/info - to get your city'
            bot.reply_to(message, help_string)
        else:
            bot.reply_to(message, '/start')
    except:
        bot.reply_to(message, 'ooopps')

bot.enable_save_next_step_handlers(delay=2)

bot.load_next_step_handlers()

bot.polling()